<a href="https://colab.research.google.com/github/camilacruz-cs/PretaLab/blob/main/mod4/%20aula2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Preparando o ambiente

In [ ]:
# Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/pasa

/content/drive/MyDrive/ciclo11_Python_Para_Dados/mod4


In [ ]:
import os
print(os.getcwd())


/content/drive/MyDrive/ciclo11_Python_Para_Dados/mod4


In [ ]:
!ls

aula1.ipynb


## Método avançados com Bibliotecas Python para Análise de Dados

In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# 1. Leitura dos dados
# Carregar o arquivo Excel enviado
file_path = '/path_to_file/pnad_2024.xlsx'  # Altere o caminho para o seu arquivo
df = pd.read_excel(file_path, sheet_name='bq-results-20241002-154627-1727')

#### Tratamento e limpeza de dados - ETL

In [ ]:
# 2. Renomear as colunas com base no dicionário fornecido
colunas_dicionario = {
    'V2007': 'sexo',
    'V2010': 'idade',
    'V4001': 'condicao_de_trabalho',
    'V4003': 'renda',
    'sigla_uf': 'estado',
    'V2009': 'raca_cor',
    'V4039': 'horas_trabalhadas'
}


In [ ]:
df_renomeado = df.rename(columns=colunas_dicionario)

In [ ]:

# 4. Convertendo as colunas para tipos apropriados
df_limpo['renda'] = pd.to_numeric(df_limpo['renda'], errors='coerce')
df_limpo['horas_trabalhadas'] = pd.to_numeric(df_limpo['horas_trabalhadas'], errors='coerce')
df_limpo['idade'] = pd.to_numeric(df_limpo['idade'], errors='coerce')


In [ ]:
 5. Adicionar colunas de interesse (CLT e cargos, assumindo que `condicao_de_trabalho` indica o tipo de contrato)
# Aqui, estou assumindo que a condição de trabalho "1" indica CLT, e "2" seria autônomo.
df_limpo['trabalhador_clt'] = df_limpo['condicao_de_trabalho'].apply(lambda x: 1 if x == 1 else 0)


#### Análise

In [ ]:
# Média de idade
media_idade = df_limpo['idade'].mean()

In [ ]:
# Filtrando pessoas com idade acima de 18 (trabalho formal)
df_maiores = df_limpo[df_limpo['idade'] >= 18]

In [ ]:

# Filtrando apenas os trabalhadores com renda maior que zero (trabalhadores ativos)
df_trabalhadores = df_maiores[df_maiores['renda'] > 0]

In [ ]:
# Média de renda e horas trabalhadas por sexo e raça/cor
df_agg_sexo_raca = df_trabalhadores.groupby(['sexo', 'raca_cor']).agg(
    media_renda=('renda', 'mean'),
    media_horas_trabalhadas=('horas_trabalhadas', 'mean'),
    qtd_trabalhadores=('sexo', 'size')
).reset_index()

In [ ]:
# Taxa de trabalhadores CLT por estado
df_clt_estado = df_trabalhadores.groupby('estado').agg(
    taxa_clt=('trabalhador_clt', 'mean'),
    media_renda=('renda', 'mean'),
    media_idade=('idade', 'mean')
).reset_index()

In [ ]:
# Criando uma coluna de "categoria de idade"
df_trabalhadores['faixa_etaria'] = pd.cut(df_trabalhadores['idade'], bins=[18, 30, 40, 50, 60, np.inf], labels=['18-30', '31-40', '41-50', '51-60', '60+'])

#### Visualização

In [ ]:
plt.figure(figsize=(14, 8))

In [ ]:
# Gráfico 1: Média de renda por faixa etária e sexo
plt.subplot(2, 2, 1)
sns.barplot(data=df_trabalhadores, x='faixa_etaria', y='renda', hue='sexo')
plt.title('Média de Renda por Faixa Etária e Sexo')
plt.xlabel('Faixa Etária')
plt.ylabel('Média de Renda')


In [ ]:
# Gráfico 2: Média de horas trabalhadas por raça/cor
plt.subplot(2, 2, 2)
sns.barplot(data=df_trabalhadores, x='raca_cor', y='horas_trabalhadas', hue='sexo')
plt.title('Média de Horas Trabalhadas por Raça/Cor')
plt.xlabel('Raça/Cor')
plt.ylabel('Média de Horas Trabalhadas')

In [ ]:
# Gráfico 3: Taxa de trabalhadores CLT por estado
plt.subplot(2, 2, 3)
sns.barplot(data=df_clt_estado, x='estado', y='taxa_clt')
plt.title('Taxa de Trabalhadores CLT por Estado')
plt.xlabel('Estado')
plt.ylabel('Taxa CLT (%)')

In [ ]:
# Gráfico 4: Distribuição de trabalhadores por faixa etária
plt.subplot(2, 2, 4)
sns.countplot(data=df_trabalhadores, x='faixa_etaria', hue='sexo')
plt.title('Distribuição de Trabalhadores por Faixa Etária')
plt.xlabel('Faixa Etária')
plt.ylabel('Contagem')

plt.tight_layout()
plt.show()

 #### Resultados Estatísticos

In [ ]:
# Exibindo os resultados estatísticos finais
print("Média de idade dos trabalhadores: ", df_trabalhadores['idade'].mean())
print("Taxa geral de trabalhadores CLT: ", df_trabalhadores['trabalhador_clt'].mean() * 100, "%")
print("Média geral de renda: ", df_trabalhadores['renda'].mean())
print("Média geral de horas trabalhadas: ", df_trabalhadores['horas_trabalhadas'].mean())


In [ ]:
# Exibindo a tabela agregada por sexo e raça/cor
print(df_agg_sexo_raca)

In [ ]:
# Exibindo a tabela de taxa CLT por estado
print(df_clt_estado)